In [45]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import progressbar
from random import choices

In [46]:
states={}
rx=-5
ro=5
empty=0
startstate="#########"
player=1
df=0.99
alpha=0.1
l=[]
c=0

In [47]:
def addkey(key,player):
    key=key+player
    states[key]=np.random.uniform(-0.01,0.01,9)
    
def checkvalidaction(key,act):
    if key[act]=='#':
        return 1
    else:
        return 0
    
def chooseaction(key,player):
    if key==startstate:
        return int(random.random()*9)
    ch=int(random.random()*2)
    if ch==1:
        while(1):
            act=int(random.random()*9)
            if checkvalidaction(key,act):
                return act
    else:
        key_t=key+player
        x=np.argsort(states[key_t])
        x=np.flip(x)
        for y in x:
            if checkvalidaction(key,y):
                return y  

def getnext(state,action,player):
    #print(state," ",action," ",player,"\n")
    l=list(state)
    l[int(action)]=player
    state=''.join(l)
    return state

def getwinner(state):
    l=list(state)
    temp=[['#' for i in range(3)] for j in range(3)]
    for i in range(3):
        for j in range(3):
            temp[i][j]=l[i*3+j]
            #print(temp)
    for i in range(3):
        if temp[i][0]==temp[i][1] and temp[i][1]==temp[i][2] and temp[i][2]!='#':
            return temp[i][0]
        if temp[0][i]==temp[1][i] and temp[1][i]==temp[2][i] and temp[0][i]!='#':
            return temp[0][i]
        
    if temp[0][0]==temp[1][1] and temp[1][1]==temp[2][2] and temp[1][1]!='#':
        return temp[0][0];
    else: 
        if temp[0][2]==temp[1][1] and temp[2][0]==temp[1][1] and temp[2][0]!='#':
            return temp[2][0]
    return 0

def getreward(state):
    winner=getwinner(state)
    if winner=='X':
        return 20
    if winner=='O':
        return -20
    return 0

def gameover(state):
    if '#' not in state:
        return 1
    if getwinner(state)!=0:
        return 1
    return 0

def gameoverlast(state):
    if '#' not in state:
        return 1
    return 0
 
def switchplayers(pl):
    if pl=='X':
        return 'O'
    else :
        return 'X'
    
def updateqvalues(state,act,ns,rw,pl):
    ex=0
    if gameoverlast(ns):
        ex=rw
    else :
        if pl=='X':
            ex=rw+(df*np.amin(states[ns+switchplayers(pl)]))
            #print(ex)
        else :
            ex=rw+(df*np.amax(states[ns+switchplayers(pl)]))
    state=state+pl
    #if alpha*(ex-states[state][act])>0.1 and alpha*(ex-states[state][act])<1:
        #print(alpha*(ex-states[state][act]))
    states[state][act]=states[state][act]+alpha*(ex-states[state][act])
    


In [48]:
def qlearn(c,maxgames):
    games=0
    player='X'
    playerop='O'
    player=int(random.random()*2)
    if player==1:
        player='X'
    else :
        player='O'
    for i in progressbar.progressbar(range(maxgames)):
        key=startstate
        cplayer=player
        while(23):
            #print(key,"\n")
            
            if key+player not in states:
                addkey(key,player)
            action=chooseaction(key,player)
            nextstate=getnext(key,action,player)
            if nextstate+switchplayers(player)=="O###O#X#XO":
                c+=1
            reward=getreward(nextstate)
            if nextstate+switchplayers(player) not in states:
                addkey(nextstate,switchplayers(player))
            updateqvalues(key,action,nextstate,reward,player)
            if gameover(nextstate)==1:
                player=switchplayers(cplayer)
                break
            else :
                key=nextstate
                player=switchplayers(player)
    return c
                        

In [49]:
c=0
states={}
c=qlearn(c,6*500000)
print(c)

 45% (1371617 of 3000000) |######        | Elapsed Time: 0:04:47 ETA:   0:05:56

KeyboardInterrupt: 

In [50]:
print(len(states))

10956


import pickle
with open("dataqp.pkl","wb") as f:
    pickle.dump(states,f)

In [51]:
def getactfromuser(state):
    while(1):
        y=int(input("ROW"))
        x=int(input("COLUMN"))
        if checkvalidaction(state,(y-1)*3+(x-1)):
            state=getnext(state,(y-1)*3+(x-1),'O')
            return state

In [52]:
def getactfromcomp(state):
    x=np.argsort(states[state+'X'])
    x=np.flip(x)
    for y in x:
        if checkvalidaction(state,y):
            state=getnext(state,y,'X')
            return state

In [53]:
def printboard(state):
    l=list(state)
    for i in range(3):
        for j in range(3):
            print(l[i*3+j]," ",end='')
        print("\n")


In [54]:
import os
from IPython.display import clear_output
def gameplay():
    state=startstate
    for i in range(5):
        state=getactfromuser(state)
        clear_output()
        printboard(state)
        print(states[state+'X'])
        if getwinner(state)=='O':
            print("You Win\n")
            return 1
        if gameover(state):
            return 1
        state=getactfromcomp(state)
        #clear_output()
        printboard(state)
        print(states[state+'O'])
        if getwinner(state)=='X':
            print("Computer wins\n")
            return 1
        

In [ ]:
gameplay()

In [41]:
import pickle
with open("dataqp.pkl","wb")as f:
    pickle.dump(states,f)